In [1]:
import boto3
import xmltodict
import json

In [2]:
environments = {
  "sandbox": {
    "endpoint": 
          "https://mturk-requester-sandbox.us-east-1.amazonaws.com",
    "preview": "https://workersandbox.mturk.com/mturk/preview"
  },
}
mturk_environment = environments["sandbox"]
session = boto3.Session(profile_name='mturk')
client = session.client(
    service_name='mturk',
    region_name='us-east-1',
    endpoint_url=mturk_environment['endpoint'],
)

In [4]:
print(client.get_account_balance()['AvailableBalance'])

10000.00


In [5]:
html_layout = open('/home/asmithh/aws_template.html', 'r').read()
QUESTION_XML = """<HTMLQuestion xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2011-11-11/HTMLQuestion.xsd">
        <HTMLContent><![CDATA[{}]]></HTMLContent>
        <FrameHeight>650</FrameHeight>
        </HTMLQuestion>"""
question_xml = QUESTION_XML.format(html_layout)


In [6]:
TaskAttributes = {
    'MaxAssignments': 5,           
    # How long the task will be available on MTurk (1 hour)     
    'LifetimeInSeconds': 10*60*60,
    # How long Workers have to complete each item (10 minutes)
    'AssignmentDurationInSeconds': 60*10,
    # The reward you will offer Workers for each response
    'Reward': '0.05',                     
    'Title': 'Provide a label for a piece of text',
    'Keywords': 'news, tagging',
    'Description': 'Choose the best tag for a piece of text.'
}

In [13]:
text = """
The narrative involved him battling an evil rival chocolate maker called "The Unknown" - a character absent from the traditional Wonka plotlines - who is trying to steal Willy's "Anti-Graffiti Gobstopper". 
"""
question = question_xml.replace('${content}', text)
tags = ['why', 'how', 'what even', 'chatgpt', 'ai is evil']
for idx, tag in enumerate(tags):
    question = question.replace('${values[' + str(idx) + ']}', tag)
    
response = client.create_hit(**TaskAttributes, Question=question)

In [11]:
hit_type_id = response['HIT']['HITTypeId']

In [14]:
hit_type_id

'334916ZQE4W7GE2U0STGIJMUDVBZTM'

In [15]:
print(mturk_environment['preview']+"?groupId={}".format(hit_type_id))

https://workersandbox.mturk.com/mturk/preview?groupId=334916ZQE4W7GE2U0STGIJMUDVBZTM


In [19]:
hit = client.get_hit(HITId=response['HIT']['HITId'])

In [20]:
hit['HIT']['HITStatus']

'Assignable'

In [21]:
assignmentsList = client.list_assignments_for_hit(
        HITId=response['HIT']['HITId'],
        AssignmentStatuses=['Submitted', 'Approved'],
        MaxResults=10
    )

In [22]:
assignmentsList

{'NextToken': 'p1:u99CVENaKOKjHeWkhdJ5/vTw0UN5+oMsmiIAkWtQ4GYyhwessrIA2c5M2ORBcg==',
 'NumResults': 1,
 'Assignments': [{'AssignmentId': '3X4MXAO0B0G93XA8HQ4ZN4UKTJKRWS',
   'WorkerId': 'AX174EFTHMXX8',
   'HITId': '3C8QQOM6K8TDCKXWXNYMCHGXMPFILX',
   'AssignmentStatus': 'Submitted',
   'AutoApprovalTime': datetime.datetime(2024, 4, 3, 13, 58, 23, tzinfo=tzlocal()),
   'AcceptTime': datetime.datetime(2024, 3, 4, 12, 58, 20, tzinfo=tzlocal()),
   'SubmitTime': datetime.datetime(2024, 3, 4, 12, 58, 23, tzinfo=tzlocal()),
   'Answer': '<?xml version="1.0" encoding="ASCII"?><QuestionFormAnswers xmlns="http://mechanicalturk.amazonaws.com/AWSMechanicalTurkDataSchemas/2005-10-01/QuestionFormAnswers.xsd"><Answer><QuestionIdentifier>best_tag</QuestionIdentifier><FreeText>how</FreeText></Answer></QuestionFormAnswers>'}],
 'ResponseMetadata': {'RequestId': 'f061f9d2-04d1-423c-955d-1b8cd4762e80',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f061f9d2-04d1-423c-955d-1b8cd4762e80',